In [ ]:
#This block to load all libraries needed
library("DEXSeq")
library("tidyr")
library("qvalue")
library("edgeR")
library("dplyr")
library("reshape2")
library("EnhancedVolcano")
library("ggplot2")
library("pheatmap")
library("org.Hs.eg.db")
library("readr")
library("purrr")
library("ggbeeswarm")
library('data.table')

In [78]:
#This block will grab each method's count files, then process all the exons to generate the venn diagram.

#grab method's files

#ExCy
Excy.countfiles = list.files('./Excy/PC1/', pattern="txt$", full.names=TRUE)

#ExoEasy
EE.countfiles = list.files('./ExoEasy/PC1/', pattern="txt$", full.names=TRUE)

#Fujifilm
FF.countfiles = list.files('./Fujifilm/PC1/', pattern="txt$", full.names=TRUE)

#UC
UC.countfiles = list.files('./UC/PC1/', pattern="txt$", full.names=TRUE)

##NEXT is the GFF
flattenedFile = list.files('.', pattern="gff$", full.names=TRUE)

###NEXT IS THE METADATA
metadata = read.csv('ExCy_metadata_dexseq.csv')
rownames(metadata) <- metadata$SampleID
metadata <- metadata[,-1]

print('Files are found, success')

####annotation files
exomega <- as.data.frame(read.csv('ExoMEGA.txt'))
ncrna_data <- read_tsv('non-coding-rna.txt', show_col_types=FALSE)
ncrna <-  ncrna_data %>% select(c(2,13))
mRNA <- read_tsv('VESICLEPEDIA_PROTEIN_MRNAS_5.1.txt')

###########Next blocks are to tabulate the count files using feature counts

##the steps in BLOCK order (increases with a # per step) for each sample block:

# grab the sample row
# read the sample row, then detect all exons in the first column for the column. 
# set the column vector to the row names
# drop the unneeded column vector
# tag all exons corresponding to the same transcript 
# filter all transcript counts less than 10 and clean up intermediate ID column
# Combine all exons corresponding to the same transcript with melt, group_by, summarize, then shift from long form to wide form data structure with dcast.


##############################################


#ExCy
Excy.metadata <- metadata[1,] 
Excy.dxd = read.table(Excy.countfiles, header=FALSE) %>% dplyr::filter(grepl("ENSG", V1, fixed=TRUE))
Excy.dxd <- Excy.dxd %>% mutate(Gene = substr(V1, 1,15))
Excy.cf <- Excy.dxd %>%  dplyr::filter(V2 > 10) %>% select(-V1)
Excy.cfs <-  melt(Excy.cf, id.vars = "Gene") %>% group_by(Gene, variable) %>% summarise(sum_value=sum(value)) %>% dcast(Gene ~ variable, value.var="sum_value")
rownames(Excy.cfs) <- Excy.cfs$Gene 


#map gene names to HUGO
annot.df <- mapIds(org.Hs.eg.db, keys = row.names(Excy.cfs), column = "SYMBOL", keytype = "ENSEMBL") %>% as.data.frame() %>% rename(SYMBOL=1)
annot.df$SYMBOL <- toupper(annot.df$SYMBOL)
Excy.cfs$gene_id <- annot.df$SYMBOL 

#filter out non-mapped genes and combine duplicate
Excy.gcfs <- Excy.cfs %>% dplyr::filter(!grepl("NA", gene_id, fixed= TRUE)) %>% select(-Gene)
Excy.gpcfs <-  melt(Excy.gcfs, id.vars = "gene_id") %>% group_by(gene_id, variable) %>% summarise(sum_value=sum(value)) %>% dcast(gene_id ~ variable, value.var="sum_value")
Excy.gpcfs <- Excy.gpcfs[complete.cases(Excy.gpcfs), ]
write.csv(Excy.gpcfs, 'Excy_before_map_venn_PC1.csv')

##Vesiclepedia mapping
Excy.exomega <- as.data.frame(Excy.gpcfs[Excy.gpcfs$gene_id %in% exomega[,1] ,])
rownames(Excy.exomega) <- Excy.exomega$gene_id
write.csv(Excy.exomega, 'Excy_after_map_PC1.csv')

#now map to mRNA
mRNA_Excy <- Excy.exomega[rownames(Excy.exomega) %in% mRNA$mRNA, ]
write.csv(mRNA_Excy, 'Excy_mRNA_PC1.csv')
Excy.em <- mRNA_Excy %>% select(-gene_id)


#######################################################################################

#EE
EE.metadata <- metadata[2,]
EE.dxd = read.table(EE.countfiles, header=FALSE) %>% dplyr::filter(grepl("ENSG", V1, fixed=TRUE))
EE.dxd <- EE.dxd %>% mutate(Gene = substr(V1, 1,15))
EE.cf <- EE.dxd %>%  dplyr::filter(V2 > 10) %>% select(-V1)
EE.cfs <-  melt(EE.cf, id.vars = "Gene") %>% group_by(Gene, variable) %>% summarise(sum_value=sum(value)) %>% dcast(Gene ~ variable, value.var="sum_value")
rownames(EE.cfs) <- EE.cfs$Gene

#map gene names to HUGO
annot.df <- mapIds(org.Hs.eg.db, keys = row.names(EE.cfs), column = "SYMBOL", keytype = "ENSEMBL") %>% as.data.frame() %>% rename(SYMBOL=1)
annot.df$SYMBOL <- toupper(annot.df$SYMBOL)
EE.cfs$gene_id <- annot.df$SYMBOL 

#filter out non-mapped genes and combine duplicate
EE.gcfs <- EE.cfs %>% dplyr::filter(!grepl("NA", gene_id, fixed= TRUE)) %>% select(-Gene)
EE.gpcfs <-  melt(EE.gcfs, id.vars = "gene_id") %>% group_by(gene_id, variable) %>% summarise(sum_value=sum(value)) %>% dcast(gene_id ~ variable, value.var="sum_value")
EE.gpcfs <- EE.gpcfs[complete.cases(EE.gpcfs), ]
write.csv(EE.gpcfs, 'EE_before_map_PC1.csv')

##Vesiclepedia mapping
EE.exomega <- as.data.frame(EE.gpcfs[EE.gpcfs$gene_id %in% exomega[,1] ,])
write.csv(EE.exomega, 'EE_after_map_PC1.csv')
rownames(EE.exomega) <- EE.exomega$gene_id
mRNA_EE <- EE.exomega[rownames(EE.exomega) %in% mRNA$mRNA ,]
write.csv(mRNA_EE, 'EE_mRNA_PC1.csv')
EE.em <- mRNA_EE %>% select(-gene_id)

#######################################################################################

#FF
FF.metadata <- metadata[3,]
FF.dxd = read.table(FF.countfiles, header=FALSE) %>% dplyr::filter(grepl("ENSG", V1, fixed=TRUE))
FF.dxd <- FF.dxd %>% mutate(Gene = substr(V1, 1,15))
FF.cf <- FF.dxd %>% dplyr::filter(V2 > 10) %>% select(-V1)
FF.cfs <-  melt(FF.cf, id.vars = "Gene") %>% group_by(Gene, variable) %>% summarise(sum_value=sum(value)) %>% dcast(Gene ~ variable, value.var="sum_value")
rownames(FF.cfs) <- FF.cfs$Gene

#map gene names to HUGO
annot.df <- mapIds(org.Hs.eg.db, keys = row.names(FF.cfs), column = "SYMBOL", keytype = "ENSEMBL") %>% as.data.frame() %>% rename(SYMBOL=1)
annot.df$SYMBOL <- toupper(annot.df$SYMBOL)
FF.cfs$gene_id <- annot.df$SYMBOL 

#filter out non-mapped genes and combine duplicate
FF.gcfs <- FF.cfs %>% dplyr::filter(!grepl("NA", gene_id, fixed= TRUE)) %>% select(-Gene)
FF.gpcfs <-  melt(FF.gcfs, id.vars = "gene_id") %>% group_by(gene_id, variable) %>% summarise(sum_value=sum(value)) %>% dcast(gene_id ~ variable, value.var="sum_value")
FF.gpcfs <- FF.gpcfs[complete.cases(FF.gpcfs), ]
write.csv(FF.gpcfs, 'FF_venn_before_map_PC1.csv')

##Vesiclepedia mapping
FF.exomega <- as.data.frame(FF.gpcfs[FF.gpcfs$gene_id %in% exomega[,1] ,])
rownames(FF.exomega) <- FF.exomega$gene_id
write.csv(FF.exomega, 'FF_after_PC1_map.csv')
mRNA_FF <- FF.exomega[rownames(FF.exomega) %in% mRNA$mRNA, ]
write.csv(mRNA_FF, 'FF_mRNA_PC1.csv')
FF.em <- mRNA_FF %>% select(-gene_id)

#######################################################################################

#UC
UC.metadata <- metadata[4,]
UC.dxd = read.table(UC.countfiles, header=FALSE) %>% dplyr::filter(grepl("ENSG", V1, fixed=TRUE))
UC.dxd <- UC.dxd %>% mutate(Gene = substr(V1, 1,15))
UC.cf <- UC.dxd %>% dplyr::filter(V2 > 10) %>% select(-V1)
UC.cfs <-  melt(UC.cf, id.vars = "Gene") %>% group_by(Gene, variable) %>% summarise(sum_value=sum(value)) %>% dcast(Gene ~ variable, value.var="sum_value")
rownames(UC.cfs) <- UC.cfs$Gene

#
annot.df <- mapIds(org.Hs.eg.db, keys = row.names(UC.cfs), column = "SYMBOL", keytype = "ENSEMBL") %>% as.data.frame() %>% rename(SYMBOL=1)
annot.df$SYMBOL <- toupper(annot.df$SYMBOL)
UC.cfs$gene_id <- annot.df$SYMBOL 

#filter out non-mapped genes and combine duplicate
UC.gcfs <- UC.cfs %>% dplyr::filter(!grepl("NA", gene_id, fixed= TRUE)) %>% select(-Gene)
UC.gpcfs <-  melt(UC.gcfs, id.vars = "gene_id") %>% group_by(gene_id, variable) %>% summarise(sum_value=sum(value)) %>% dcast(gene_id ~ variable, value.var="sum_value")
UC.gpcfs <- UC.gpcfs[complete.cases(UC.gpcfs), ]
write.csv(UC.gpcfs, 'UC_venn_before_map_PC1.csv')

##Vesiclepedia mapping
UC.exomega <- as.data.frame(UC.gpcfs[UC.gpcfs$gene_id %in% exomega[,1] ,])
rownames(UC.exomega) <- UC.exomega$gene_id
write.csv(UC.exomega, 'UC_after_map_PC1.csv')
mRNA_UC <- UC.exomega[rownames(UC.exomega) %in% mRNA$mRNA, ]
write.csv(mRNA_UC, 'UC_venn_PC1.csv')
UC.em <- mRNA_UC %>% select(-gene_id)

print('Samples exons are found and have been successfuly processed for venn diagram analysis \n please see the csv to do the venn diagram analysis')

[1] "Files are found, success"


Rows: 9989 Columns: 1
-- Column specification --------------------------------------------------------
Delimiter: "\t"
chr (1): mRNA

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
`summarise()` has grouped output by 'Gene'. You can override using the
`.groups` argument.
'select()' returned 1:many mapping between keys and columns

`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'Gene'. You can override using the
`.groups` argument.
'select()' returned 1:many mapping between keys and columns

`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.
`summarise()` has grouped output by 'Gene'. You can override using the
`.groups` argument.
'select()' returned 1:many mapping between keys and columns

`summarise()` has grouped output by 'gene_id'. You can override usin

[1] "Samples exons are found and have been successfuly processed for venn diagram analysis \n please see the csv to do the venn diagram analysis"


In [80]:
##THis block of code is to generate the RNA biotype distributions using the GENCODE database (as a txt file import)####

#Load GENCODE by file
ncrna_data <- read_tsv('non-coding-rna.txt', show_col_types=FALSE)
ncrna <-  ncrna_data %>% select(c(2,13))



#Block write the columns to do the dictionary filter

##Make the column
ExCy.exomega_filt <- cbind(Excy.gpcfs, Genes=Excy.gpcfs$'gene_id')
FF.exomega_filt <- cbind(FF.gpcfs, Genes=FF.gpcfs$'gene_id')
UC.exomega_filt <- cbind(UC.gpcfs, Genes=UC.gpcfs$'gene_id')
EE.exomega_filt <- cbind(EE.gpcfs, Genes=EE.gpcfs$'gene_id')


##Compute teh lengths of each list to generate the total RNA count
EE_len <- length(EE.exomega_filt$Genes)
FF_len <- length(FF.exomega_filt$Genes)
UC_len <- length(UC.exomega_filt$Genes)
ExCy_len <- length(ExCy.exomega_filt$Genes)


#Do the filtering against GENCODE
EE_filt <- ncrna[ncrna$'Approved symbol' %in% EE.exomega_filt$Genes ,]
FF_filt <- ncrna[ncrna$'Approved symbol' %in% FF.exomega_filt$Genes ,]
UC_filt <- ncrna[ncrna$'Approved symbol' %in% UC.exomega_filt$Genes ,]
ExCy_filt <- ncrna[ncrna$'Approved symbol' %in% ExCy.exomega_filt$Genes ,]


# Initialize the summary dataframe
summary_df <- data.frame(count = numeric(0))

# Define dataframes and lengths
dataframes <- list(ExCy_filt, EE_filt, FF_filt, UC_filt)
lengths <- list(ExCy_len, EE_len, FF_len, UC_len)
methods <- list('Excy','ExoEasy','Fujifilm','UC')

#Biotypes to look for:
keywords <- c("Antisense", "Long intergenic", "Intronic", "Long non-coding RNAs", "Small nuclear RNAs", 
              'transfer RNAs ', 'Divergent', 'MicroRNA', 'Mitochondrially', 'Nuclear-encoded mitochondrial',
              'Overlapping transcripts', 'Piwi-interacting', 'Ribosomal','Ro60-associated Y', 'Small Cajal body-specific',
              'Small NF90', 'Small nucleolar RNA', 'Variant U1', 'Vault'
              )

# Loop through dataframes
for (i in seq_along(dataframes)) {
  current_df <- dataframes[[i]]
  current_len <- lengths[[i]]
  current_method <- methods[[i]]
    
  # Loop through keywords and calculate counts and percentages
  for (keyword in keywords) {
    df_counts <- current_df %>%
      filter(grepl(keyword, `Group name`, ignore.case = TRUE)) %>%
      summarise(count = n()) %>%
      mutate(Subgroup = keyword)
    
    # Append to summary_df
    summary_df <- summary_df %>%
      bind_rows(df_counts) 
  }
  
  # Append length to summary_df
  summary_df <- rbind(summary_df, 'Length' = current_len) 
  
  # Print the method and corresponding result
  print(current_method)  
  print(summary_df)
 
    
  # Reinitialize summary_df for the next iteration
  summary_df <- data.frame(count = numeric(0))
}

###########TOTAL RNA DISTRIBUTION AFTER MAPPING
ExCy.exomega_filt <- cbind(Excy.exomega, Genes=rownames(Excy.exomega))
FF.exomega_filt <- cbind(FF.exomega, Genes=rownames(FF.exomega))
UC.exomega_filt <- cbind(UC.exomega, Genes=rownames(UC.exomega))
EE.exomega_filt <- cbind(EE.exomega, Genes=rownames(EE.exomega))



EE_len <- length(EE.exomega_filt$Genes)
FF_len <- length(FF.exomega_filt$Genes)
UC_len <- length(UC.exomega_filt$Genes)
ExCy_len <- length(ExCy.exomega_filt$Genes)
#length(rownames(EE.exomega))

EE_filt <- ncrna[ncrna$'Approved symbol' %in% EE.exomega_filt$Genes ,]
FF_filt <- ncrna[ncrna$'Approved symbol' %in% FF.exomega_filt$Genes ,]
UC_filt <- ncrna[ncrna$'Approved symbol' %in% UC.exomega_filt$Genes ,]
ExCy_filt <- ncrna[ncrna$'Approved symbol' %in% ExCy.exomega_filt$Genes ,]

# Initialize the summary dataframe
summary_df <- data.frame(count = numeric(0))

# Define dataframes and lengths
dataframes <- list(ExCy_filt, EE_filt, FF_filt, UC_filt)
lengths <- list(ExCy_len, EE_len, FF_len, UC_len)
methods <- list('Excy','ExoEasy','Fujifilm','UC')

#Biotypes to look for:
keywords <- c("Antisense", "Long intergenic", "Intronic", "Long non-coding RNAs", "Small nuclear RNAs", 
              'transfer RNAs ', 'Divergent', 'MicroRNA', 'Mitochondrially', 'Nuclear-encoded mitochondrial',
              'Overlapping transcripts', 'Piwi-interacting', 'Ribosomal','Ro60-associated Y', 'Small Cajal body-specific',
              'Small NF90', 'Small nucleolar RNA', 'Variant U1', 'Vault'
              )

# Loop through dataframes
for (i in seq_along(dataframes)) {
  current_df <- dataframes[[i]]
  current_len <- lengths[[i]]
  current_method <- methods[[i]]
    
  # Loop through keywords and calculate counts and percentages
  for (keyword in keywords) {
    df_counts <- current_df %>%
      filter(grepl(keyword, `Group name`, ignore.case = TRUE)) %>%
      summarise(count = n()) %>%
      mutate(Subgroup = keyword)
    
    # Append to summary_df
    summary_df <- summary_df %>%
      bind_rows(df_counts) 
  }
  
  # Append length to summary_df
  summary_df <- rbind(summary_df, 'Length' = current_len) 
  
  # Print the method and corresponding result
  print(current_method)  
  print(summary_df)
 
    
  # Reinitialize summary_df for the next iteration
  summary_df <- data.frame(count = numeric(0))
}




[1] "Excy"
   count                      Subgroup
1    683                     Antisense
2   1137               Long intergenic
3     31                      Intronic
4    174          Long non-coding RNAs
5      8            Small nuclear RNAs
6      0                transfer RNAs 
7    136                     Divergent
8    375                      MicroRNA
9      0               Mitochondrially
10     0 Nuclear-encoded mitochondrial
11     7       Overlapping transcripts
12     0              Piwi-interacting
13     0                     Ribosomal
14     1             Ro60-associated Y
15     0     Small Cajal body-specific
16     0                    Small NF90
17    86           Small nucleolar RNA
18     3                    Variant U1
19     0                         Vault
20 21788                         21788
[1] "ExoEasy"
   count                      Subgroup
1    653                     Antisense
2   1049               Long intergenic
3     24                      Intronic


In [88]:
########THis block of code is to generate the heatmap and correlations shown in the main figure and supplement

#Same as before, except when dealing with multiple files, now we can use  feature counts and DEXSeq to grab all the files 
# read the sample rows, then detect all exons in the first column for the column. 
# set the column vector to the row names
# drop the unneeded column vector
# tag all exons corresponding to the same transcript 
# filter all transcript counts less than 10 and clean up intermediate ID column
# Combine all exons corresponding to the same transcript with melt, group_by, summarize, then shift from long form to wide form data structure with dcast.

#grab file
All.countfiles=list.files('./ALL_PC1/', pattern="txt$", full.names=TRUE)


##Generate count file
All.dxd = DEXSeqDataSetFromHTSeq(countfiles = All.countfiles, sampleData = metadata[1:4,], design = ~ sample + exon, flattenedfile = flattenedFile)
All.count <- as.data.frame(featureCounts(All.dxd))

#Do the light filtering and aggregation
All.count$ID <- rownames(All.count)
All.count <- All.count %>% mutate(Gene = substr(ID, 1,15))
All.cf <- All.count %>% mutate(row_sum = rowSums(.[,-5:-6])) %>% dplyr::filter(row_sum > 10) %>% select(-ID)
All.cfs <-  melt(All.cf, id.vars = "Gene") %>% group_by(Gene, variable) %>% summarise(sum_value=sum(value)) %>% dcast(Gene ~ variable, value.var="sum_value")
rownames(All.cfs) <- All.cfs$Gene

#prepare for annotation
exomega <- as.data.frame(read.csv('ExoMEGA.txt'))
mRNA <- read_tsv('VESICLEPEDIA_PROTEIN_MRNAS_5.1.txt')

##map gene names to HUGO
annot.df <- mapIds(org.Hs.eg.db, keys = row.names(All.cfs), column = "SYMBOL", keytype = "ENSEMBL") %>% as.data.frame() %>% rename(SYMBOL=1)
annot.df$SYMBOL <- toupper(annot.df$SYMBOL)
All.cfs$gene_id <- annot.df$SYMBOL 

#filter out non-mapped genes and combine duplicate
All.gcfs <- All.cfs %>% dplyr::filter(!grepl("NA", gene_id, fixed= TRUE)) %>% select(-Gene)
All.gpcfs <-  melt(All.gcfs, id.vars = "gene_id") %>% group_by(gene_id, variable) %>% summarise(sum_value=sum(value)) %>% dcast(gene_id ~ variable, value.var="sum_value")
All.gpcfs <- All.gpcfs[complete.cases(All.gpcfs), ]

All.exomega <- as.data.frame(All.gpcfs[All.gpcfs$gene_id %in% exomega[,1] ,])
rownames(All.exomega) <- All.exomega$gene_id
mRNA_All <- All.exomega[rownames(All.exomega) %in% mRNA$mRNA ,]

All.exomega <- mRNA_All[,2:5]

###
print("All files have been collated together, successfully processed, and ready for analysis")
print("All files have been collated together, successfully processed, and ready for analysis")
###


#######ANALYSIS####

##Sample correlation

#Change names
colnames(All.exomega) <- c("ExCy","ExoEasy","Fujifilm","UC")
All.corr <- cor(All.exomega)
All.crdy <- round(log2(All.corr+1), 2)


pheatmap(All.crdy, 
         fontsize_row=24, 
         fontsize_col=24, 
         display_numbers=T, 
         fontsize_number=24, 
         border_color="white", 
         number_color="black", 
         legend_labels=c("0.75","0.80","0.85","0.90","0.95","1.00"), 
         legend_breaks=c(0.75,0.8,0.85,0.90,0.95,1), 
         color=colorRampPalette(c("pink","red"))(6), 
         cluster_cols = FALSE, 
         cluster_rows=FALSE, 
         filename="corr_PC1.png")


#COSMIC HEATMAP 
COSMIC <- as.data.frame(read.csv('COSMIC.txt'))
colnames(All.exomega) <- c("ExCy","ExoEasy","Fujifilm","UC")


#Key in cosmic pancreas genes to Matrix
All.cosmic <- as.data.frame(All.exomega[rownames(All.exomega) %in% COSMIC[,1] ,])

pheatmap(log2(All.cosmic+1), 
         scale='row', 
         border_color="white", 
         fontsize=30, 
         number_color="black", 
         display_numbers=F, 
         fontface="bold", 
         filename="cosmic_heatmap_PC1.png", 
         angle_col=0, 
         width=11, 
         height=9)

####THIS BLOCK IS FOR THE BEEPLOT
beeplot <- log2(All.exomega+1) %>% cbind(., Genes=rownames(.)) %>% pivot_longer(-Genes, names_to = "Method", values_to = "Expression")

#color vector
colours <- c("ExCy" ="red3", "ExoEasy"="orange", "Fujifilm"="grey", "UC"="blue2")


#plotting
options(repr.plot.width = 9, repr.plot.height =5)


ggplot(beeplot, aes(x = Method, y = Expression, color = Method)) +
  geom_quasirandom(cex=0.5, width=0.5) +
  scale_color_manual(values=colours)+
  labs(x = NULL, y = "Log2 mRNA Transcripts") +
  theme_classic() +
    theme(axis.line = element_line(color='black'),
    plot.background = element_blank(),
    panel.grid.major = element_blank(),
    panel.grid.minor = element_blank(),
    axis.text.x = element_text(size=25, face="bold", color="black"),
    axis.text.y = element_text(size=25, color="black"),
    text = element_text(size=25, face='bold')
          )

converting counts to integer mode

Warning message in DESeqDataSet(rse, design, ignoreRank = TRUE):
"some variables in design formula are characters, converting to factors"
Warning message:
"attributes are not identical across measure variables; they will be dropped"
`summarise()` has grouped output by 'Gene'. You can override using the
`.groups` argument.
Rows: 9989 Columns: 1
-- Column specification --------------------------------------------------------
Delimiter: "\t"
chr (1): mRNA

i Use `spec()` to retrieve the full column specification for this data.
i Specify the column types or set `show_col_types = FALSE` to quiet this message.
'select()' returned 1:many mapping between keys and columns

`summarise()` has grouped output by 'gene_id'. You can override using the
`.groups` argument.


[1] "All files have been collated together, successfully processed, and ready for analysis"
[1] "All files have been collated together, successfully processed, and ready for analysis"


In [ ]:
####THIS NEXT BLOCK IS FOR TCGA HEATMAP RELATIVE TO COSMIC.THIS BLOCK TAKES A LONG TIME TO PROCESS, SO WE LEAVE IT AS A
#SEPARATE BLOCK. 

###MAKE SURE YOU EXTRACT THE TCGA WINZIP INTO THE TCGA FOLDER WITHIN THE DIRECTORY

#Load TCGA files
TCGA_files <- list.files('./TCGA/', pattern="tsv$", full.names=TRUE)

#seed starter. The seed starter initiates collating, then we loop all the files found.
seed <- fread(TCGA_files[1], select=c("gene_name", "unstranded")) %>% slice(-c(1:4))

#this is our loop
for (files in TCGA_files) {
    
    #read a file from TCGA
    file <- fread(files, select=c("unstranded")) %>% slice(-c(1:4))
    seed <- cbind(seed, file)
}

#This is the final collated dataframe, LOAD ONLY WITH HEAD.
TCGA <- seed

#Key in cosmic pancreas genes to Matrix
TCGA_COSMIC <- as.data.frame(TCGA[TCGA$gene_name %in% COSMIC[,1] ,]) 
rownames(TCGA_COSMIC) <- TCGA_COSMIC$gene_name 
TCGA_COSMIC <- TCGA_COSMIC[2:length(TCGA_COSMIC)]
names(TCGA_COSMIC) <- paste0("Patient_",seq_along(TCGA_COSMIC)) 
#write.csv(TCGA_COSMIC, 'TCGA_Cosmic.csv')


#Now for the heatmap
pheatmap(log2(TCGA_COSMIC+1), 
         border_color="white",
         scale="row",
         fontsize=15, 
         number_color="black", 
         display_numbers=F, 
         fontface="bold",
         show_colnames=FALSE,
         cluster_rows=TRUE,
         filename="TCGA_PC1.png", 
         angle_col=0, 
         width=6, 
         height=5,
         breaks=seq(-1, 1, length.out=101))